In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/text_matching/chinese/main')

In [2]:
%tensorflow_version 1.x
!pip install bert4keras

TensorFlow 1.x selected.
  Created wheel for bert4keras: filename=bert4keras-0.7.7-cp36-none-any.whl size=36804 sha256=b90c43df7f212f29d7049f8435ee19ffd93fbace107502f2939835980c98b6b4
  Stored in directory: /root/.cache/pip/wheels/73/5f/2b/ee7932b73172503fdb429421c0b433f47a737c0d1c2a85fdd2
Successfully built bert4keras


In [3]:
import numpy as np
import csv

from bert4keras.backend import keras, set_gelu, K
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Dropout, Dense

Using TensorFlow backend.


In [0]:
def load_data(filename):
    D = []
    with open(filename, encoding='utf-8') as f:
      for i, line in enumerate(csv.reader(f, delimiter=',')):
        if i == 0:
          continue
        text1, text2, label = line
        D.append((text1, text2, int(label)))
    return D


class data_generator(DataGenerator):
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (text1, text2, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(
                text1, text2, max_length=maxlen
            )
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []


def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total


class Evaluator(keras.callbacks.Callback):
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(test_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            # you can save your model here
        print(
            'val_acc: {:.4f}, best_val_acc: {:.4f}'.format
            (val_acc, self.best_val_acc)
        )

In [5]:
set_gelu('tanh')
maxlen = 64
batch_size = 32
config_path = '../model/chinese_wwm_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '../model/chinese_wwm_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '../model/chinese_wwm_L-12_H-768_A-12/vocab.txt'

train_data = load_data('../data/train.csv')
test_data = load_data('../data/test.csv')

tokenizer = Tokenizer(dict_path, do_lower_case=True)

train_generator = data_generator(train_data, batch_size)
test_generator = data_generator(test_data, batch_size)

bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    with_pool=True,
    return_keras_model=False,
)

output = Dropout(rate=.1)(bert.model.output)
output = Dense(
    units=2, activation='softmax', kernel_initializer=bert.initializer
)(output)

model = keras.models.Model(bert.model.input, output)
model.summary()

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(2e-5),
    metrics=['accuracy'],
)

evaluator = Evaluator()
model.fit_generator(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=20,
    callbacks=[evaluator]
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
___________